![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [1365]:
import pandas as pd
import numpy as np
# Start coding here...

In [1366]:
# Open csv file and take a overview
data = pd.read_csv("bank_marketing.csv")

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [1367]:
# Spliting the data 
client_df = data[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [1368]:
# Spliting the data
campaign_df = data[['client_id', 'campaign','month','day', 'duration', 'pdays', 'previous', 'poutcome', 'y']]

,client_id,campaign,month,day,duration,pdays,previous,poutcome,y
0,0,1,may,13,261,999,0,nonexistent,no
1,1,1,may,19,149,999,0,nonexistent,no
2,2,1,may,23,226,999,0,nonexistent,no
3,3,1,may,27,151,999,0,nonexistent,no
4,4,1,may,3,307,999,0,nonexistent,no


In [1369]:
# Spliting the data
economics_df = data[['client_id','emp_var_rate', 'cons_price_idx', 'euribor3m','nr_employed']]
economics_df.head()

,client_id,emp_var_rate,cons_price_idx,euribor3m,nr_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [1394]:
# Showing the columns
print("client dataframe: \n", client_df.columns, '\n')
print("campaign dataframe: \n", campaign_df.columns, '\n')
print("economics dataframe: \n", economics_df.columns, '\n')

client dataframe: 
 Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object') 

campaign dataframe: 
 Index(['campaign_id', 'client_id', 'number_contacts', 'contact_duration',
       'pdays', 'previous_campaign_contacts', 'previous_outcome',
       'campaign_outcome', 'last_contact_date'],
      dtype='object') 

economics dataframe: 
 Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object') 



In [1370]:
# Change columns' name, client
client_df.rename(columns={'client_id':'id'},inplace=True)

In [1371]:
# Change columns' name, campaign
campaign_df.rename(columns={'duration':'contact_duration',
                    'previous':'previous_campaign_contacts', 
                    "y":"campaign_outcome",
                    "poutcome":"previous_outcome",
                    "campaign":"number_contacts"}, inplace=True)

In [1372]:
# Change columns' name, economics
economics_df.rename(columns={"euribor3m":"euribor_three_months",
                     "nr_employed":"number_employed"},inplace=True)


In [1373]:
# Clening education column from client
client_df['education'] = client_df['education'].str.replace(".","_")
client_df['education'] = client_df['education'].replace('unknown',np.nan)

In [1374]:
# Cleaning job column from client
client_df['job'] = client_df['job'].str.replace(".","")


array(['housemaid', 'services', 'admin', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [1395]:
# Checking the columns
print(client_df['education'].unique(), "\n")
print(client_df['job'].unique(), "\n")

['basic_4y' 'high_school' 'basic_6y' 'basic_9y' 'professional_course' nan
 'university_degree' 'illiterate'] 

['housemaid' 'services' 'admin' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student'] 



In [1375]:
# Convert previus_outcome column and checking
campaign_df['previous_outcome']=\
            campaign_df['previous_outcome']\
            .replace("success",1)

campaign_df['previous_outcome']=\
            campaign_df['previous_outcome']\
            .replace("failure", 0)

campaign_df['previous_outcome']=\
            campaign_df['previous_outcome']\
            .replace("nonexistent", np.nan)

campaign_df.previous_outcome.unique()

array([nan,  0.,  1.])

In [1376]:
# Converting in campaign_outcome column and checking
campaign_df['campaign_outcome']=\
            campaign_df['campaign_outcome']\
            .replace("yes",1)

campaign_df['campaign_outcome']=\
            campaign_df['campaign_outcome']\
            .replace("no", 0)

campaign_df.campaign_outcome.unique()

array([0, 1])

In [1377]:
# Some creations for a mode difficult date modeling, but don't neaded
'''
    bool_date=campaign_df['pdays']==999
    campaign_df['']
    campaign_df.loc(bool_date,)
    campaign_df['last_contact_date']=pd.to_datetime(data[['month','day','']])
'''

In [1378]:
# Add campaign_id on campaign dataframe
campaign_df.insert(0,'campaign_id',1)
campaign_df.columns

Index(['campaign_id', 'client_id', 'number_contacts', 'month', 'day',
       'contact_duration', 'pdays', 'previous_campaign_contacts',
       'previous_outcome', 'campaign_outcome'],
      dtype='object')

In [1379]:
# Create a temp dataframe:date_df
date_df=pd.DataFrame()
date_df['month']=campaign_df['month']
date_df['day']=campaign_df['day']

KeyError: 'month'

In [1380]:
# Inserting the year column on date_df
date_df.insert(0, 'year',2022)
date_df.columns

ValueError: cannot insert year, already exists

In [1381]:
# Convert month column strings to approiaate numbers
date_df['month'].unique()
date_df['month']=date_df['month'].replace({'jun':6,
                                          'jul':7,
                                          'may':5, 
                                          'aug':8, 
                                          'oct':10,
                                          'nov':11,
                                          'dec':12,
                                          'mar':3,
                                          'apr':4,
                                          'sep':9})
date_df['month'].unique()


array([ 5,  6,  7,  8, 10, 11, 12,  3,  4,  9])

In [1382]:
# Creating the last_contact_date column
campaign_df['last_contact_date']=pd.to_datetime(date_df[['year','month','day']])
campaign_df['last_contact_date']

0       2022-05-13
1       2022-05-19
2       2022-05-23
3       2022-05-27
4       2022-05-03
           ...    
41183   2022-11-30
41184   2022-11-06
41185   2022-11-24
41186   2022-11-17
41187   2022-11-23
Name: last_contact_date, Length: 41188, dtype: datetime64[ns]

In [1383]:
# Checking redundancy
campaign_df.columns
del campaign_df['month']
del campaign_df['day']
campaign_df.columns

Index(['campaign_id', 'client_id', 'number_contacts', 'contact_duration',
       'pdays', 'previous_campaign_contacts', 'previous_outcome',
       'campaign_outcome', 'last_contact_date'],
      dtype='object')

In [1384]:
#checking redundacy
client_df.head()


,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [1385]:
#checking redundacy
economics_df.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [1386]:
# Save all three DataFrame
client_df.to_csv('client.csv',index=False)
campaign_df.to_csv('campaign.csv',index=False)
economics_df.to_csv('economics.csv',index=False)


In [1387]:
# Create table client
client_table = """CREATE TABLE client
(
        id serial PRIMARY KEY,
        age integer,
        job text,
        marital text,
        education text,
        credit_default boolean,
        housing boolean,
        loan boolean
);
\COPY client FROM 'client.csv' DELIMITER ',' CSV HEADER
"""


In [1388]:
# Create table campaign
campaign_table = """CREATE TABLE campaign
(
        campaign_id serial PRIMARY KEY,
        client_id serial REFERENCES client (id),
        number_contacts integer,
        contact_duration integer,
        pdays integer,
        previous_campaign_contacts integer,
        previous_outcome boolean,
        campaign_outcome boolean,
        last_contact_date date
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""


In [1389]:
# Create table economics
economics_table = """CREATE TABLE economics
(
        client_id serial REFERENCES client (id),
        emp_var_rate float,
        cons_price_idx float,
        euribor_three_months float,
        number_employed float
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""
